# Dive into Deep Learning
# Ch 6. Builders’ Guide

 In this chapter, we will peel back the curtain, digging deeper into the key components of deep learning computation, namely model construction, parameter access and initialization, designing custom layers and blocks, reading and writing models to disk, and leveraging GPUs to achieve dramatic speedups. These insights will move you from end user to power user, giving you the tools needed to reap the benefits of a mature deep learning library while retaining the flexibility to implement more complex models, including those you invent yourself! 

이 장에서는 장막을 벗기고 딥 러닝 계산의 핵심 구성 요소, 즉 모델 구성, 매개 변수 액세스 및 초기화, 사용자 지정 레이어 및 블록 설계, 디스크에 모델 읽기 및 쓰기, 극적인 속도 향상을 달성하기 위한 GPU 활용에 대해 더 깊이 파고들 것입니다. 이러한 통찰력은 최종 사용자에서 고급 사용자로 이동하여 성숙한 딥 러닝 라이브러리의 이점을 얻는 데 필요한 도구를 제공하는 동시에 직접 발명한 모델을 포함하여 더 복잡한 모델을 구현할 수 있는 유연성을 유지합니다!


# Layers and Modules ( 레이어 및 모듈 )

In this section, we will explore the concept of neural network modules and their significance in constructing complex neural networks. Initially, we focused on linear models with a single output, represented by a single neuron. However, as we moved towards networks with multiple outputs, we leveraged vectorized arithmetic to describe entire layers of neurons.

본 세션에서는 신경망 모듈의 개념과 복잡한 신경망을 구축하는 중요성에 대해 살펴보겠습니다. 초기에는 단일 뉴런으로 표현되는 단일 출력을 갖는 선형 모델에 초점을 맞췄습니다. 그러나 여러 출력을 갖는 네트워크로 전환함에 따라 벡터화된 산술 연산을 사용하여 여러 뉴런으로 이루어진 레이어를 설명했습니다.

For multi-layer perceptrons (MLPs), both the entire model and its constituent layers share a similar structure. The model takes raw inputs, generates predictions, and has parameters from all layers. Each layer receives inputs from the previous layer, generates outputs for the subsequent layer, and has tunable parameters updated based on backward flow of signals.

다중 레이어 퍼셉트론(MLPs)의 경우 전체 모델과 구성 요소 레이어가 유사한 구조를 가집니다. 모델은 원시 입력을 받아 예측을 생성하며 모든 레이어의 매개변수를 가지고 있습니다. 각 레이어는 이전 레이어로부터 입력을 받아 다음 레이어를 위한 출력을 생성하며, 역방향 신호 흐름에 따라 조정 가능한 매개변수를 업데이트합니다.

While neurons, layers, and models are important abstractions, it is often convenient to speak about components larger than an individual layer but smaller than the entire model. For instance, popular computer vision architecture, ResNet-152, consists of hundreds of layers organized in repeating patterns of groups of layers. Implementing such networks one layer at a time can be tedious. Therefore, we introduce the concept of neural network modules, which can describe a single layer, a component with multiple layers, or the entire model itself.

뉴런, 레이어 및 모델은 중요한 추상화 요소이지만 종종 개별 레이어보다는 크고 전체 모델보다 작은 구성 요소에 대해 이야기하는 것이 편리합니다. 예를 들어, 인기 있는 컴퓨터 비전 아키텍처인 ResNet-152는 수백 개의 레이어로 구성된 반복 패턴의 레이어 그룹으로 구성됩니다. 이러한 네트워크를 하나의 레이어씩 구현하는 것은 지루할 수 있습니다. 따라서 우리는 신경망 모듈의 개념을 소개하고, 개별 레이어, 여러 레이어로 이루어진 구성 요소, 또는 전체 모델을 설명할 수 있습니다.

![](http://d2l.ai/_images/blocks.svg)

> Multiple layers are combined into modules, forming repeating patterns of larger models.<br>
여러 레이어가 모듈로 결합되어 더 큰 모델의 반복 패턴을 형성합니다.

The advantage of using neural network modules is their ability to be combined into larger artifacts, often recursively. This allows for compact code and easy implementation of complex neural networks. Modules are represented as classes, and any subclass must define a forward propagation method to transform input into output and store necessary parameters. Notably, some modules may not require any parameters. Furthermore, the modules must have a backpropagation method for gradient calculation, facilitated by the auto differentiation feature.

신경망 모듈을 사용하는 장점은 이러한 모듈을 더 큰 모듈로 결합하여 반복적으로 사용할 수 있다는 점입니다. 이로 인해 복잡한 신경망을 간단한 코드로 쉽게 구현할 수 있습니다. 모듈은 클래스로 표현되며, 모든 서브클래스는 입력을 출력으로 변환하는 순전파 방법과 필요한 매개변수를 저장해야 합니다. 특히 일부 모듈은 매개변수를 필요로 하지 않을 수 있습니다. 또한 모듈은 자동 미분 기능으로 가능한 그라디언트 계산을 위한 역전파 방법을 가져야 합니다.

Through the use of neural network modules, we gain greater flexibility in designing intricate neural network architectures, enabling us to tackle a wide array of tasks in various domains, such as computer vision, natural language processing, and speech. The power of this approach is exemplified by the success of architectures like ResNet, which have achieved state-of-the-art results in multiple competitions.

신경망 모듈을 사용함으로써 우리는 복잡한 신경망 아키텍처를 설계하는 데 더 큰 유연성을 얻으며, 컴퓨터 비전, 자연어 처리, 음성 등 다양한 영역에서 다양한 작업을 수행할 수 있습니다. 이러한 접근 방식의 성과는 ResNet과 같은 아키텍처가 여러 대회에서 최고 성적을 달성한 데에서도 확인할 수 있습니다.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

/home/nplab/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


To begin, we revisit the code that we used to implement MLPs. The following code generates a network with one fully connected hidden layer with 256 units and ReLU activation, followed by a fully connected output layer with 10 units (no activation function).

시작하려면 MLP를 구현하는 데 사용한 코드를 다시 살펴보겠습니다. 다음 코드는 256개의 단위와 ReLU 활성화가 있는 완전히 연결된 하나의 숨겨진 계층과 10개의 단위(활성화 함수 없음)가 있는 완전히 연결된 출력 계층이 있는 네트워크를 생성합니다.

In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

/home/nplab/.local/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

In this example, we constructed our model by instantiating an nn.Sequential, with layers in the order that they should be executed passed as arguments. In short, nn.Sequential defines a special kind of Module, the class that presents a module in PyTorch. It maintains an ordered list of constituent Modules. Note that each of the two fully connected layers is an instance of the Linear class which is itself a subclass of Module. The forward propagation (forward) method is also remarkably simple: it chains each module in the list together, passing the output of each as input to the next. Note that until now, we have been invoking our models via the construction net(X) to obtain their outputs. This is actually just shorthand for net.__call__(X).

이 예제에서는 nn.Sequential을 인스턴스화하여 모델을 구성했습니다. 실행해야 할 레이어들을 인자로 전달하여 순서대로 정의합니다. 간단히 말해, nn.Sequential은 PyTorch에서 모듈을 나타내는 특별한 종류의 Module을 정의합니다. 이는 구성 요소 모듈들의 순서가 유지되는 목록을 유지합니다. 두 개의 완전 연결 레이어 각각은 Linear 클래스의 인스턴스입니다. Linear 클래스 자체도 Module의 하위 클래스입니다. forward propagation (forward) 메서드 또한 매우 간단합니다. 각 모듈을 목록으로 연결하고, 각 모듈의 출력을 다음 모듈의 입력으로 전달합니다. 지금까지는 모델을 net(X) 구성을 통해 호출하여 출력을 얻었습니다. 실제로 이것은 net.call(X)의 약식으로 사용된 것입니다.


##  A Custom Module

To gain a better understanding of how modules work, let's implement one ourselves. Before proceeding, let's summarize the basic functionality that each module should provide:

모듈 작동 방식을 더 잘 이해하기 위해 모듈을 직접 구현해 봅시다. 진행하기 전에 각 모듈이 제공해야 하는 기본 기능을 요약해 보겠습니다.

1. Ingest input data as arguments to its forward propagation method.
    - 정방향 전파 방법에 대한 인수로 입력 데이터를 수집합니다.
2. Generate an output by having the forward propagation method return a value. Note that the output may have a different shape from the input. For example, the first fully connected layer in our model above ingests an input of arbitrary dimension but returns an output of dimension 256.
    - 정방향 전파 방법이 값을 반환하도록 하여 출력을 생성합니다. 출력은 입력과 모양이 다를 수 있습니다. 예를 들어 위 모델의 첫 번째 완전 연결 레이어는 임의 차원의 입력을 수집하지만 차원 256의 출력을 반환합니다.
3. Calculate the gradient of its output with respect to its input, which can be accessed via its backpropagation method. Typically this happens automatically.
    - 역전파 방법을 통해 액세스할 수 있는 입력에 대한 출력 기울기를 계산합니다. 일반적으로 이것은 자동으로 발생합니다.
4. Store and provide access to those parameters necessary to execute the forward propagation computation.
    - 정방향 전파 계산을 실행하는 데 필요한 매개변수를 저장하고 액세스를 제공합니다.
5. Initialize model parameters as needed.
    - 필요에 따라 모델 매개변수를 초기화합니다.


Below is the code for a custom module representing an MLP with one hidden layer containing 256 hidden units and a 10-dimensional output layer. The MLP class inherits from the nn.Module class, and it implements its constructor (__init__ method) and the forward propagation method.

아래는 256개의 히든 유닛과 10차원 출력 레이어를 포함하는 하나의 히든 레이어가 있는 MLP를 나타내는 사용자 정의 모듈의 코드입니다. MLP 클래스는 nn.Module 클래스에서 상속되며 해당 생성자(__init__ 메서드) 및 정방향 전파 메서드를 구현합니다.

In [3]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

The forward propagation method takes input X, calculates the hidden representation with an activation function applied, and outputs the logits. The layers (self.hidden and self.out) are instance variables of the MLP class. This design allows us to create multiple instances of MLPs, each representing a distinct learned model.

정방향 전파 방법은 입력 X를 취하고 활성화 함수가 적용된 숨겨진 표현을 계산하고 로짓을 출력합니다. 레이어(self.hidden 및 self.out)는 MLP 클래스의 인스턴스 변수입니다. 이 설계를 통해 각각 고유한 학습 모델을 나타내는 여러 MLP 인스턴스를 만들 수 있습니다.

In the constructor, we instantiate the layers (self.hidden and self.out) and invoke them in the forward propagation method. We utilize super().__init__() to call the parent class's constructor, avoiding the need to repeat boilerplate code for most modules. The backpropagation method and parameter initialization will be automatically generated, relieving us from implementing these functionalities.

생성자에서 레이어(self.hidden 및 self.out)를 인스턴스화하고 순방향 전파 방법에서 호출합니다. 우리는 부모 클래스의 생성자를 호출하기 위해 super().__init__()를 활용하여 대부분의 모듈에 대한 상용구 코드를 반복할 필요가 없습니다. 역전파 방법과 매개변수 초기화가 자동으로 생성되어 이러한 기능을 구현하지 않아도 됩니다.

Let's create an instance of the MLP and observe its output shape:

MLP의 인스턴스를 만들고 출력 형태를 관찰해 보겠습니다.

In [4]:
net = MLP()
net(X).shape

torch.Size([2, 10])

The module abstraction is highly versatile, allowing us to subclass it to create layers (e.g., fully connected layers), entire models (like the MLP above), or intermediate components of varying complexity. This versatility will be valuable in the subsequent chapters, particularly when dealing with convolutional neural networks.

모듈 추상화는 매우 다재다능하여 계층(예: 완전히 연결된 계층), 전체 모델(위의 MLP와 같은) 또는 다양한 복잡성의 중간 구성 요소를 생성하기 위해 하위 클래스를 만들 수 있습니다. 이 다양성은 특히 컨볼루션 신경망을 다룰 때 다음 장에서 유용할 것입니다.

## The Sequential Module ( 순차 모듈 )

We can now take a closer look at how the `Sequential` class works. Recall that `Sequential` was designed to daisy-chain other modules together. To build our own simplified `MySequential`, we just need to define two key methods:

이제 Sequential 클래스가 어떻게 작동하는지 자세히 살펴볼 수 있습니다. Sequential은 다른 모듈을 함께 데이지 체인 방식으로 연결하도록 설계되었습니다. 단순화된 자체 MySequential을 구축하려면 다음 두 가지 주요 메서드를 정의하기만 하면 됩니다.

1. A method to append modules one by one to a list.
    - 목록에 모듈을 하나씩 추가하는 방법입니다.
2. A forward propagation method to pass an input through the chain of modules, in the same order as they were appended.
    - 추가된 것과 동일한 순서로 모듈 체인을 통해 입력을 전달하는 정방향 전파 방법입니다.

The following `MySequential` class delivers the same functionality of the default `Sequential` class.

다음 `MySequential` 클래스는 기본 `Sequential` 클래스와 동일한 기능을 제공합니다.

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In the __init__ method, we add every module by calling the add_modules method. These modules can be accessed by the children method later. In this way the system knows the added modules, and it will properly initialize each module’s parameters.

__init__ 메서드에서 add_modules 메서드를 호출하여 모든 모듈을 추가합니다. 이러한 모듈은 나중에 children 메소드로 액세스할 수 있습니다. 이러한 방식으로 시스템은 추가된 모듈을 알고 각 모듈의 매개변수를 적절하게 초기화합니다.

When our MySequential’s forward propagation method is invoked, each added module is executed in the order in which they were added. We can now reimplement an MLP using our MySequential class.

MySequential의 정방향 전파 방법이 호출되면 추가된 각 모듈이 추가된 순서대로 실행됩니다. 이제 MySequential 클래스를 사용하여 MLP를 다시 구현할 수 있습니다.

In [6]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

## Executing Code in the Forward Propagation Method ( 정방향 전파 방법으로 코드 실행 )

The `Sequential` class makes model construction easy, allowing us to assemble new architectures without having to define our own class. However, not all architectures are simple daisy chains. When greater flexibility is required, we will want to define our own blocks. For example, we might want to execute Python's control flow within the forward propagation method. Moreover, we might want to perform arbitrary mathematical operations, not simply relying on predefined neural network layers. 

'Sequential' 클래스는 모델 구성을 쉽게 만들어 자체 클래스를 정의할 필요 없이 새로운 아키텍처를 조립할 수 있도록 합니다. 그러나 모든 아키텍처가 단순한 데이지 체인인 것은 아닙니다. 더 큰 유연성이 필요한 경우 자체 블록을 정의할 수 있습니다. 예를 들어 정방향 전파 방법 내에서 Python의 제어 흐름을 실행하려고 할 수 있습니다. 또한 미리 정의된 신경망 계층에 단순히 의존하는 것이 아니라 임의의 수학적 연산을 수행하기를 원할 수도 있습니다.

You might have noticed that until now, all of the operations in our networks have acted upon our network's activations and its parameters. Sometimes, however, we might want to incorporate terms that are neither the result of previous layers nor updatable parameters. We call these *constant parameters*.  Say for example that we want a layer that calculates the function $f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$, where $\mathbf{x}$ is the input, $\mathbf{w}$ is our parameter, and $c$ is some specified constant that is not updated during optimization. So we implement a `FixedHiddenMLP` class as follows.

지금까지 네트워크의 모든 작업이 네트워크의 활성화 및 해당 매개 변수에 따라 작동했음을 알 수 있습니다. 그러나 때로는 이전 레이어의 결과도 아니고 업데이트 가능한 매개변수도 아닌 용어를 통합해야 할 수 있습니다. 이러한 *상수 매개변수*라고 합니다. 예를 들어 $f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$ 함수를 계산하는 레이어가 필요하다고 가정해 보겠습니다. 여기서 $\mathbf{x}$는 입력이고 $\mathbf{w}$는 매개변수이며 $c$는 최적화 중에 업데이트되지 않는 지정된 상수입니다. 그래서 다음과 같이 `FixedHiddenMLP` 클래스를 구현합니다.

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In this FixedHiddenMLP model, we implement a hidden layer whose weights (self.rand_weight) are initialized randomly at instantiation and are thereafter constant. This weight is not a model parameter and thus it is never updated by backpropagation. The network then passes the output of this “fixed” layer through a fully connected layer.

이 FixedHiddenMLP 모델에서 가중치(self.rand_weight)가 인스턴스화 시 무작위로 초기화되고 그 이후에는 일정하게 유지되는 숨겨진 레이어를 구현합니다. 이 가중치는 모델 매개변수가 아니므로 역전파에 의해 업데이트되지 않습니다. 그런 다음 네트워크는 이 "고정" 계층의 출력을 완전히 연결된 계층을 통해 전달합니다.

Note that before returning the output, our model did something unusual. We ran a while-loop, testing on the condition its $\ell_1$ norm is larger than $1$, and dividing our output vector by $2$ until it satisfied the condition. Finally, we returned the sum of the entries in `X`. To our knowledge, no standard neural network performs this operation. Note that this particular operation may not be useful in any real-world task. Our point is only to show you how to integrate arbitrary code into the flow of your neural network computations.

출력을 반환하기 전에 모델이 비정상적인 작업을 수행했습니다. 우리는 $\ell_1$ 규범이 $1$보다 큰 조건에서 테스트하고 조건을 만족할 때까지 출력 벡터를 $2$로 나누는 while 루프를 실행했습니다. 마지막으로 `X` 항목의 합계를 반환했습니다. 우리가 아는 한 표준 신경망은 이 작업을 수행하지 않습니다. 이 특정 작업은 실제 작업에서는 유용하지 않을 수 있습니다. 우리의 요점은 임의의 코드를 신경망 계산 흐름에 통합하는 방법을 보여주는 것뿐입니다.

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.0221, grad_fn=<SumBackward0>)

We can mix and match various ways of assembling modules together. In the following example, we nest modules in some creative ways.

모듈을 함께 조립하는 다양한 방법을 혼합하고 일치시킬 수 있습니다. 다음 예에서는 몇 가지 창의적인 방식으로 모듈을 중첩합니다.

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.0273, grad_fn=<SumBackward0>)

# Parameter Management ( 매개변수 관리 )


After choosing the architecture and setting hyperparameters, we proceed to the training loop to minimize the loss function and find optimal parameter values. These parameters are needed for future predictions and can be extracted for reuse, model saving, or examination to gain scientific understanding.

아키텍처를 선택하고 하이퍼파라미터를 설정한 후 학습 루프로 진행하여 손실 함수를 최소화하고 최적의 파라미터 값을 찾습니다. 이러한 매개변수는 향후 예측에 필요하며 과학적 이해를 얻기 위해 재사용, 모델 저장 또는 검사를 위해 추출할 수 있습니다.

While deep learning frameworks handle most parameter-related complexities, for non-standard architectures, we might need to delve into declaring and manipulating parameters. In this section, we cover:

딥 러닝 프레임워크는 대부분의 매개변수 관련 복잡성을 처리하지만 비표준 아키텍처의 경우 매개변수 선언 및 조작을 자세히 살펴봐야 할 수도 있습니다. 이 섹션에서는 다음을 다룹니다.

1. Accessing parameters for debugging, diagnostics, and visualizations.
    - 디버깅, 진단 및 시각화를 위한 매개변수 액세스.
2. Sharing parameters across different model components.
    - 서로 다른 모델 구성요소 간에 매개변수 공유.
    
    
We start by focusing on an MLP with one hidden layer.

하나의 숨겨진 계층이 있는 MLP에 초점을 맞추는 것으로 시작합니다.

In [10]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## Parameter Access ( 매개변수 액세스 )

When a model is defined via the Sequential class, we can first access any layer by indexing into the model as though it were a list. Each layer’s parameters are conveniently located in its attribute.

모델이 Sequential 클래스를 통해 정의되면 목록인 것처럼 모델에 인덱싱하여 모든 레이어에 먼저 액세스할 수 있습니다. 각 레이어의 매개변수는 해당 속성에 편리하게 위치합니다.

We can inspect the parameters of the second fully connected layer as follows.

다음과 같이 두 번째 완전 연결 계층의 매개변수를 검사할 수 있습니다.

In [11]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.1059, -0.2030, -0.2031, -0.1254, -0.1964, -0.1672,  0.0802, -0.2192]])),
             ('bias', tensor([-0.2558]))])

We can see that this fully connected layer contains two parameters, corresponding to that layer’s weights and biases, respectively.

이 완전히 연결된 계층에는 해당 계층의 가중치와 편향에 각각 해당하는 두 개의 매개변수가 포함되어 있음을 알 수 있습니다.

### 1. Targeted Parameters ( 대상 매개변수 )

Note that each parameter is represented as an instance of the parameter class. To do anything useful with the parameters, we first need to access the underlying numerical values. There are several ways to do this. Some are simpler while others are more general. The following code extracts the bias from the second neural network layer, which returns a parameter class instance, and further accesses that parameter’s value.

각 매개변수는 매개변수 클래스의 인스턴스로 표시됩니다. 매개변수로 유용한 작업을 수행하려면 먼저 기본 숫자 값에 액세스해야 합니다. 이를 수행하는 방법에는 여러 가지가 있습니다. 일부는 더 간단하고 다른 일부는 더 일반적입니다. 다음 코드는 매개변수 클래스 인스턴스를 반환하는 두 번째 신경망 계층에서 편향을 추출하고 해당 매개변수의 값에 추가로 액세스합니다.

In [12]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.2558]))

Parameters are complex objects, containing values, gradients, and additional information. That is why we need to request the value explicitly.

매개변수는 값, 기울기 및 추가 정보를 포함하는 복잡한 개체입니다. 그렇기 때문에 값을 명시적으로 요청해야 합니다.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

값 외에도 각 매개변수를 통해 기울기에 액세스할 수 있습니다. 아직 이 네트워크에 대한 역전파를 호출하지 않았기 때문에 초기 상태에 있습니다.

In [13]:
net[2].weight.grad == None

True

### 2. All Parameters at Once ( 한 번에 모든 매개변수 )

When we need to perform operations on all parameters, accessing them one-by-one can grow tedious. The situation can grow especially unwieldy when we work with more complex modules (e.g., nested modules), since we would need to recurse through the entire tree to extract each sub-module’s parameters. Below we demonstrate accessing the parameters of all layers.

모든 매개변수에 대해 작업을 수행해야 하는 경우 하나씩 액세스하는 것이 지루할 수 있습니다. 더 복잡한 모듈(예: 중첩 모듈)로 작업할 때 상황이 특히 다루기 힘들 수 있습니다. 각 하위 모듈의 매개변수를 추출하기 위해 전체 트리를 통해 재귀해야 하기 때문입니다. 아래에서는 모든 레이어의 매개변수에 액세스하는 방법을 보여줍니다.

In [14]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## Tied Parameters

Often, we want to share parameters across multiple layers. Let’s see how to do this elegantly. In the following we allocate a fully connected layer and then use its parameters specifically to set those of another layer. Here we need to run the forward propagation net(X) before accessing the parameters.

종종 여러 레이어에서 매개변수를 공유하려고 합니다. 이를 우아하게 수행하는 방법을 살펴보겠습니다. 다음에서는 완전히 연결된 계층을 할당한 다음 해당 매개변수를 사용하여 다른 계층의 매개변수를 설정합니다. 여기서 매개변수에 액세스하기 전에 순방향 전파 net(X)를 실행해야 합니다.

In [15]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters of the second and third layer are tied. They are not just equal, they are represented by the same exact tensor. Thus, if we change one of the parameters, the other one changes, too.

이 예는 두 번째 및 세 번째 레이어의 매개변수가 연결되어 있음을 보여줍니다. 그것들은 단지 같지 않고 동일한 정확한 텐서로 표현됩니다. 따라서 매개변수 중 하나를 변경하면 다른 매개변수도 변경됩니다.

You might wonder, when parameters are tied what happens to the gradients? Since the model parameters contain gradients, the gradients of the second hidden layer and the third hidden layer are added together during backpropagation.

매개변수가 연결되면 그라디언트에 어떤 일이 발생하는지 궁금할 수 있습니다. 모델 매개변수에 그래디언트가 포함되어 있으므로 역전파 과정에서 두 번째 은닉층과 세 번째 은닉층의 그래디언트가 함께 추가됩니다.

# Parameter Initialization ( 매개변수 초기화 ) 


The deep learning framework provides default random initializations to its layers. However, we often want to initialize our weights according to various other protocols. The framework provides most commonly used protocols, and also allows to create a custom initializer.

딥 러닝 프레임워크는 레이어에 기본 임의 초기화를 제공합니다. 그러나 우리는 종종 다양한 다른 프로토콜에 따라 가중치를 초기화하기를 원합니다. 프레임워크는 가장 일반적으로 사용되는 프로토콜을 제공하며 사용자 지정 초기화 프로그램을 만들 수도 있습니다.

By default, PyTorch initializes weight and bias matrices uniformly by drawing from a range that is computed according to the input and output dimension. PyTorch’s nn.init module provides a variety of preset initialization methods.

기본적으로 PyTorch는 입력 및 출력 차원에 따라 계산되는 범위에서 그려 가중치 및 편향 행렬을 균일하게 초기화합니다. PyTorch의 nn.init 모듈은 다양한 사전 설정 초기화 방법을 제공합니다.

In [16]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## Built-in Initialization ( 내장 초기화 )

Let’s begin by calling on built-in initializers. The code below initializes all weight parameters as Gaussian random variables with standard deviation 0.01, while bias parameters cleared to zero.

내장 이니셜라이저를 호출하여 시작하겠습니다. 아래 코드는 모든 가중치 매개변수를 표준 편차가 0.01인 가우시안 확률 변수로 초기화하고 바이어스 매개변수는 0으로 초기화합니다.

In [17]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0054,  0.0068, -0.0057,  0.0032]), tensor(0.))

It also initializes all parameters to the given constant value (e.g. 1).

또한 모든 매개변수를 주어진 상수 값(예: 1)으로 초기화합니다.

In [18]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

It also applies different initializers for specific blocks. For example, below we initialize the first layer with the Xavier initializer and the second layer with the constant value 42.

또한 특정 블록에 대해 다른 초기화 프로그램을 적용합니다. 예를 들어 아래에서 Xavier 초기화로 첫 번째 레이어를 초기화하고 상수 값 42로 두 번째 레이어를 초기화합니다.

In [19]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.4671,  0.5291,  0.0145,  0.1368])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### Custom Initialization

Sometimes, the initialization methods we need are not provided by the deep learning framework. In the example below, we define an initializer for any weight parameter $w$ using the following strange distribution:

때때로 필요한 초기화 방법이 딥 러닝 프레임워크에서 제공되지 않습니다. 아래 예제에서는 다음과 같은 이상한 분포를 사용하여 가중치 매개변수 $w$에 대한 초기화 프로그램을 정의합니다.


$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Again, implement my_init function to apply to net.

다시 my_init 함수를 구현하여 net에 적용합니다.

In [20]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -0.0000,  0.0000,  0.0000],
        [ 9.3774,  9.5909, -6.4113,  9.8004]], grad_fn=<SliceBackward0>)

Note that we always have the option of setting parameters directly.

항상 매개변수를 직접 설정할 수 있는 옵션이 있습니다.

In [21]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

# Custom Layers ( 사용자 지정 레이어 )

One factor behind deep learning’s success is the availability of a wide range of layers that can be composed in creative ways to design architectures suitable for a wide variety of tasks. For instance, researchers have invented layers specifically for handling images, text, looping over sequential data, and performing dynamic programming. Sooner or later, you will encounter or invent a layer that does not exist yet in the deep learning framework. In these cases, you must build a custom layer. In this section, we show you how.

딥 러닝의 성공 뒤에 있는 한 가지 요인은 다양한 작업에 적합한 아키텍처를 설계하기 위해 창의적인 방식으로 구성할 수 있는 광범위한 레이어를 사용할 수 있다는 것입니다. 예를 들어 연구원들은 이미지, 텍스트 처리, 순차 데이터 반복 및 동적 프로그래밍 수행을 위해 특별히 레이어를 발명했습니다. 조만간 딥 러닝 프레임워크에 아직 존재하지 않는 계층을 만나거나 발명하게 될 것입니다. 이러한 경우 사용자 정의 계층을 작성해야 합니다. 이 섹션에서는 방법을 보여줍니다.

##  Layers without Parameters ( 매개변수가 없는 레이어 ) 

To start, we construct a custom layer that does not have any parameters of its own. The following CenteredLayer class simply subtracts the mean from its input. To build it, we simply need to inherit from the base layer class and implement the forward propagation function

시작하려면 자체 매개변수가 없는 사용자 지정 레이어를 구성합니다. 다음 CenteredLayer 클래스는 단순히 입력에서 평균을 뺍니다. 빌드하려면 기본 레이어 클래스에서 상속하고 순방향 전파 기능을 구현하기만 하면 됩니다.

In [22]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

Let’s verify that our layer works as intended by feeding some data through it.

계층을 통해 일부 데이터를 공급하여 계층이 의도한 대로 작동하는지 확인하겠습니다.

In [23]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can now incorporate our layer as a component in constructing more complex models.

이제 더 복잡한 모델을 구성하는 구성 요소로 레이어를 통합할 수 있습니다.

In [24]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

As an extra sanity check, we can send random data through the network and check that the mean is in fact 0. Because we are dealing with floating point numbers, we may still see a very small nonzero number due to quantization.

추가 온전성 검사로 네트워크를 통해 임의의 데이터를 보내고 평균이 실제로 0인지 확인할 수 있습니다. 부동 소수점 숫자를 다루기 때문에 양자화로 인해 여전히 0이 아닌 매우 작은 숫자를 볼 수 있습니다.

In [25]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-2.0955e-09, grad_fn=<MeanBackward0>)

## Layers with Parameters ( 매개변수가 있는 레이어 )

We can use built-in functions to create parameters, which provide some basic housekeeping functionality. In particular, they govern access, initialization, sharing, saving, and loading model parameters. This way, among other benefits, we will not need to write custom serialization routines for every custom layer.

내장 함수를 사용하여 몇 가지 기본 정리 기능을 제공하는 매개변수를 만들 수 있습니다. 특히 액세스, 초기화, 공유, 저장 및 로드 모델 매개변수를 제어합니다. 이렇게 하면 다른 이점 중에서도 모든 사용자 지정 레이어에 대해 사용자 지정 직렬화 루틴을 작성할 필요가 없습니다.

Now let’s implement our own version of the fully connected layer. Recall that this layer requires two parameters, one to represent the weight and the other for the bias. In this implementation, we bake in the ReLU activation as a default. This layer requires two input arguments: in_units and units, which denote the number of inputs and outputs, respectively.

이제 완전히 연결된 계층의 자체 버전을 구현해 보겠습니다. 이 레이어에는 두 개의 매개변수가 필요하다는 점을 기억하세요. 하나는 가중치를 나타내고 다른 하나는 편향을 나타냅니다. 이 구현에서는 ReLU 활성화를 기본값으로 굽습니다. 이 레이어에는 두 개의 입력 인수가 필요합니다. in_units와 units는 각각 입력과 출력의 수를 나타냅니다.

In [26]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

Next, we instantiate the MyLinear class and access its model parameters.

다음으로 MyLinear 클래스를 인스턴스화하고 해당 모델 매개변수에 액세스합니다.

In [27]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.0513,  0.3512, -0.6121],
        [ 1.2042,  1.7937, -1.2847],
        [-0.7559,  0.5357,  0.9648],
        [ 0.4416,  0.2411, -0.2083],
        [-1.1117,  0.0412,  1.0256]], requires_grad=True)

We can directly carry out forward propagation calculations using custom layers.

커스텀 레이어를 사용하여 순방향 전파 계산을 직접 수행할 수 있습니다.

In [28]:
linear(torch.rand(2, 5))

tensor([[0.9482, 3.4843, 0.0000],
        [0.0000, 2.1634, 0.3244]])

We can also construct models using custom layers. Once we have that we can use it just like the built-in fully connected layer.

사용자 지정 레이어를 사용하여 모델을 구성할 수도 있습니다. 일단 가지고 있으면 내장된 완전히 연결된 레이어처럼 사용할 수 있습니다.

In [29]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library. Once defined, custom layers can be invoked in arbitrary contexts and architectures. Layers can have local parameters, which can be created through built-in functions.

기본 레이어 클래스를 통해 사용자 정의 레이어를 디자인할 수 있습니다. 이를 통해 라이브러리의 기존 레이어와 다르게 동작하는 유연한 새 레이어를 정의할 수 있습니다. 일단 정의되면 임의의 컨텍스트 및 아키텍처에서 사용자 지정 레이어를 호출할 수 있습니다. 레이어는 내장 함수를 통해 생성할 수 있는 로컬 매개변수를 가질 수 있습니다.